# Estudiante : Jeisson Estevens Araque Ramirez
# Asignatura : Programación para análisis de datos
# Docente: Diego Leon Zapata Ruiz
### Evidencia de aprendizaje 1. Análisis y herramientas de extracción de datos
--- 

In [1]:
# Importar las librerias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Web scraping con selenium

In [2]:
opts = Options()

opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36")
# Configuración para no abrir el navegador
opts.add_argument("--headless")


# Configurar el controlador web
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=opts
) 

# Abre la página de Mercado Libre
driver.get("https://listado.mercadolibre.com.co/")

# Espera a que se cargue la página
time.sleep(3)

# Buscar "smart watch" en la barra de búsqueda
print("Buscando en el navegador")
search_bar = driver.find_element(By.NAME, "as_word")
search_bar.send_keys("smart-watch")
search_bar.send_keys(Keys.RETURN)

# Espera a que se carguen los resultados
time.sleep(3)

# Lista para almacenar los datos
watch_data = []

# Extrae los primeros resultados de "smart watch"
print("Extrayendo resultados")
products = driver.find_elements(By.XPATH, "//li[contains(@class, 'ui-search-layout__item')]")
for product in products[:20]:  # Limitar a los primeros 20 resultados
    try:
        # Extraer el nombre del producto usando la clase proporcionada
        name = product.find_element(By.CLASS_NAME, "poly-component__title").text
        
        # Extraer el precio usando la clase correspondiente
        try:
            price = product.find_element(By.CLASS_NAME, "andes-money-amount").text
        except:
            price = "No disponible"
        
        # Añadir datos a la lista
        watch_data.append({"Nombre": name, "Precio": price})
    except Exception as e:
        print("Error al obtener los detalles del producto:", e)

# Cierra el navegador
driver.quit()

# Crear un DataFrame con los datos extraídos
print("Creando dataframe")
df_selenium = pd.DataFrame(watch_data)
df_selenium['Nombre'] = df_selenium['Nombre'].str.replace(',', '')
# Limpiar la columna de precios eliminando el carácter '\n' y cualquier espacio extra
df_selenium['Precio'] = df_selenium['Precio'].str.replace('\n', '').str.strip()

# Guardar el DataFrame en un archivo CSV
df_selenium.to_csv("smart_watch_mercadolibre_selenium.csv", index=False, encoding="utf-8", sep=",")

print(df_selenium)

print("***** Terminado *******")

Buscando en el navegador
Extrayendo resultados
Creando dataframe
***** Terminado *******


In [3]:
# Visualizar el dataframe
df_selenium


,Nombre,Precio
0,Brazalete Amazfit Gts 4 Mini Smartwatch 1.65 c...,$428.900
1,Reloj Inteligente Smartwatch Serie 8 T900 Pro ...,$66.444
2,Xiaomi Redmi Watch 5 Active Con Hyperos Bateri...,$203.900
3,Smartwatch Reloj Inteligente Deportivo Linkon ...,$159.990
4,Smartwatch Reloj Inteligente Deportivo Linkon ...,$199.990
5,Xiaomi Redmi Watch 5 Active Con Hyperos Bateri...,$203.900
6,Smartwatch Xiaomi Redmi Watch 5 Lite Amoled 1....,$278.197
7,Redmi Watch 4 Silver Gray - Gris,$336.287
8,Reloj Inteligente Ip67 Impermeable Con Linterna,$223.000
9,Reloj Inteligente Para Mujer Con Llamada Bt Y ...,$159.000


In [ ]:
# Filtrar el dataframe por la marca que nos interese
print("Filtro de Xiaomi")
print(df_selenium[df_selenium["Nombre"].str.contains("Xiaomi")])

# Beatifulsoup

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

print("****** Iniciando ******** ")

# Define la URL de búsqueda en Mercado Libre
url = "https://listado.mercadolibre.com.co/smart-watch"

# Realiza la solicitud a la página
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)

# Analiza el contenido HTML de la página
soup = BeautifulSoup(response.text, 'html.parser')

# Listas para almacenar los datos
names = []
prices = []

# Encuentra todos los contenedores de productos
print("Listando productos")
products = soup.find_all("li", class_="ui-search-layout__item")

# Extrae el nombre y precio de cada producto
for product in products[:20]:  # Limitamos a los primeros 20 resultados
    # Extrae el nombre del producto
    name_tag = product.find("h2", class_="poly-component__title")
    if name_tag:
        names.append(name_tag.text.strip())

    # Extrae el precio del producto
    price_tag = product.find("span", class_="andes-money-amount")
    if price_tag:
        prices.append(price_tag.text.strip())

# Crea un DataFrame con los datos extraídos
print("Ceando dataframe...")
df_bs4 = pd.DataFrame({
    "Nombre": names,
    "Precio": prices
})

df_bs4['Nombre'] = df_bs4['Nombre'].str.replace(',', '')

print("Creando archivo")

df_bs4.to_csv("smart_watch_mercadolibre.csv", index=False, encoding="utf-8")

print(df_bs4)

print("***** Terminado *****")


****** Iniciando ******** 
Listando productos
Ceando dataframe...
Creando archivo
***** Terminado *****


In [5]:
# Muestra el DataFrame
df_bs4

,Nombre,Precio
0,Pulsera Inteligente Xiaomi Mi Band 9 Version G...,$189.890
1,Reloj Inteligente Para Mujer Con Llamada Bt Y ...,$159.000
2,Xiaomi Redmi Watch 5 Active Con Hyperos Bateri...,$149.900
3,Xiaomi Redmi Watch 5 Active Con Hyperos Bateri...,$149.900
4,Smartwatch Xiaomi Redmi Watch 5 Lite Amoled 1....,$232.500
5,Redmi Watch 4 Silver Gray - Gris,$336.287
6,Samsung Galaxy Fit 3 Smartwatch Sport Gris Osc...,$199.900
7,Reloj inteligente Colmi P71 con pantalla de 19...,$87.500
8,Smart Watch T900 Promax L Serie 8 2.0 Color Negro,$49.900
9,Genérica t800 ultra Negro Smartwatch display ...,$32.900
